# Exploring Flow*

In this blog post we [...]

## Installing the tool

We first download the tool version 2.1.0 (May 2017) from the webpage https://flowstar.org/dowloads/

The installation process is simple provided that the required dependencies are met.

We installed Flow* in two different machines:

- Ubuntu LTS v20.04 running on a Windows desktop machine (Asus ZenBook UX331UA-AS51).
- Dell XPS15 running Linux Fedora 34.

In Fedora, we used `dnf install` to install the required dependencies `gmp-devel`, `gsl-devel`, `glpk-devel`, `flex` and `bison`. The library `mpfr` was downloaded and compiled using the instructions in the [downloads page](https://www.mpfr.org/mpfr-current/).

The required dependencies are detailed in Flow*'s [user manual](https://home.cs.colorado.edu/~xich8622/manual/manual-2.0.0.pdf).

## Solving the Lotka-Volterra model

Apart from the models that can be downloaded in the [Flow* webpage](https://flowstar.org/), we are aware of an online model repository from the [*Theory of Hybrid Systems Group* from RWTH Aachen University](https://ths.rwth-aachen.de/research/projects/hypro/benchmarks-of-continuous-and-hybrid-systems/).

We use the Lotka-Volterra model because it is a simple model. For comparison, the same model can be found in the documentation ReachabilityAnalysis.jl [here](https://juliareach.github.io/ReachabilityAnalysis.jl/dev/models/LotkaVolterra/). 

The model file is copied next. It can be read as a plain text file.

```
continuous reachability
{
 state var x,y
	
 setting
 {
  fixed steps 0.02
  time 5
  remainder estimation 1e-5
  QR precondition
  gnuplot octagon x,y
  adaptive orders { min 4 , max 6 }
  cutoff 1e-20
  precision 53
  output Lotka_Volterra
  print on
 }
	
 poly ode 2
 {
  x' = 1.5*x - x*y
  y' = -3*y + x*y
 }
 
 init
 {
  x in [4.8 , 5.2]
  y in [1.8 , 2.2]
 }
}
```

The model file describes a purely continuous model (`continuous reachability`), then the names for the state variables are defined (`state var x,y`).

The next part defines the settings for reachability analysis (`setting`):

- `fixed steps 0.02`: use fixed time steps of size `0.02`
- `time 5`: time horizon for the simulation
- `remainder estimation 1e-5`: estimate of the remainder before each iteration
- `QR precondition`: preconditioning technique for the Taylor expansion
- `gnuplot octagon x,y`: overapproximation method for plotting
- `adaptive orders { min 4 , max 6 }`: range of Taylor model's order
- `cutoff 1e-20`: threshold for polynomial coefficients (
- `precision 53`: precision (number of bits) for the interval bounds
- `output Lotka_Volterra`: number of output file
- `print on`: verbose on

The following block of code defines the differential equations; in this case,

```
 poly ode 2
 {
  x' = 1.5*x - x*y
  y' = -3*y + x*y
 }
```

Where `poly ode 2` specifies a system with two variables.

Finally, the `init` block specify the settings for reachability analysis. 

```julia
 init
 {
  x in [4.8 , 5.2]
  y in [1.8 , 2.2]
 }
```

In this case this defines a hyperrectangle of sides `4.8 .. 5.2` and `1.8 .. 2.2`.

## Running the example

We made a small modification in the model that can be downloaded from 

[ ..... ]

## Parsing the result

In [118]:
src = "LotkaVolterra/outputs/Lotka_Volterra.flow"

"LotkaVolterra/outputs/Lotka_Volterra.flow"

In [119]:
text = String(read(src))
flow = split(text, "{");

flow = flow[3:end]

@show N = length(flow) # number of reach-sets

R = flow[1] # first reach-set
data = split(R, "\n")
filter!(!isempty, data) # saco los que estan vacios
pop!(data) # saco el ultimo "}"
data

N = length(flow) = 500


5-element Vector{SubString{String}}:
 "x = [5.00000000000000e0 , 5.00" ⋯ 2446 bytes ⋯ "0625e-8 , 1.70313210117098e-8]"
 "y = [2.00000000000000e0 , 2.00" ⋯ 2452 bytes ⋯ "2587e-8 , 1.77595808728572e-8]"
 "local_t in [0 , 1.00000000000001e-2]"
 "local_var_1 in [-1.00000000000000e0 , 1.00000000000000e0]"
 "local_var_2 in [-1.00000000000000e0 , 1.00000000000000e0]"

In [110]:
data_x = data[1] # "x" variable

"x = [5.00000000000000e0 , 5.00000000000000e0] + [2.00000000000000e-1 , 2.00000000000001e-1] * local_var_1 + [-2.50000000000000e0 , -2.50000000000000e0] * local_t + [-1.00000000000001e0 , -1.00000000000000e0] * local_t * local_var_2 + [-1.00000000000001e-1 , -1.000000000" ⋯ 1966 bytes ⋯ "cal_t^4 * local_var_1^2 + [-8.41718750000027e0 , -8.41718749999971e0] * local_t^5 * local_var_2 + [5.42578124999991e0 , 5.42578125000012e0] * local_t^5 * local_var_1 + [-3.93610026041663e1 , -3.93610026041660e1] * local_t^6 + [-1.43728770330625e-8 , 1.70313210117098e-8]"

In [111]:
using TaylorModels

In [112]:
1 .. 2 # define un intervalo (si bien imprime como [1, 2])

[1, 2]

In [113]:
[1, 2] # define un array (no interesa)

2-element Vector{Int64}:
 1
 2

In [115]:
# quedarme con el right-hand side
data_x = split(data_x, "=")[2] |> strip

# renombrar variables
data_x = replace(data_x, "local_var_1" => "x")
data_x = replace(data_x, "local_var_2" => "y")
data_x = replace(data_x, "local_t" => "t")

# renombrar intervalos
data_x = replace(data_x, "," => "..")
data_x = replace(data_x, "[" => "(")
data_x = replace(data_x, "]" => ")")

data_x

LoadError: BoundsError: attempt to access 1-element Vector{SubString{String}} at index [2]

The aim is to convert this reach-set to a `TaylorModelN` whose coefficients are intervals.

In [116]:
using TaylorModels

In [117]:
t, x, y = set_variables("t x y", order=8)

3-element Vector{TaylorN{Float64}}:
  1.0 t + 𝒪(‖x‖⁹)
  1.0 x + 𝒪(‖x‖⁹)
  1.0 y + 𝒪(‖x‖⁹)

In [20]:
R = Meta.parse(data_x) |> eval

 [4.99999, 5.00001] + [-2.5, -2.5] t + [0.199999, 0.200001] x + [-9.375, -9.375] t² + [-0.100001, -0.0999999] t x + [-1.00001, -1] t y + [2.39583, 2.39584] t³ + [-1.37501, -1.375] t² x + [-0.500001, -0.5] t² y + [-0.0400001, -0.0399999] t x y + [20.7421, 20.7422] t⁴ + [-0.570834, -0.570833] t³ x + [3.79166, 3.79167] t³ y + [-0.0400001, -0.0399999] t² x² + [-0.120001, -0.119999] t² x y + [0.0999999, 0.100001] t² y² + [7.93619, 7.9362] t⁵ + [4.30885, 4.30886] t⁴ x + [4.67708, 4.67709] t⁴ y + [-0.0933334, -0.0933333] t³ x² + [0.351666, 0.351667] t³ x y + [0.316666, 0.316667] t³ y² + [-0.00400001, -0.00399999] t² x² y + [0.00399999, 0.00400001] t² x y² + [-39.3611, -39.361] t⁶ + [5.42578, 5.42579] t⁵ x + [-8.41719, -8.41718] t⁵ y + [0.230833, 0.230834] t⁴ x² + [1.28499, 1.28501] t⁴ x y + [-0.345834, -0.345833] t⁴ y² + [-0.00266667, -0.00266666] t³ x³ + [0.00133333, 0.00133334] t³ x² y + [0.0393333, 0.0393334] t³ x y² + [-0.00666667, -0.00666666] t³ y³ + 𝒪(‖x‖⁹)

In [21]:
typeof(R)

TaylorN{Interval{Float64}}

In [23]:
eval(Meta.parse("1+1"))

2

- Taylor1
- TaylorN
- TaylorModel1
- TaylorModelN

Vemos que el resto del modelo de taylor aparece al final de la expresion.

In [79]:
aux = split(data_x, "+")
rem = aux[end]
rem = Meta.parse(rem) |> eval # resto del modelo de Taylor

[-1.43729e-08, 1.70314e-08]

In [80]:
split("A + B + C", "+") # observar que el + no lo cuenta

3-element Vector{SubString{String}}:
 "A "
 " B "
 " C"

In [107]:
# le saco la parte del remainder (el +1 es para que tambien elimine el simbolo +)
q = length(data_x) - (length(aux[end]) + 1)
pol = data_x[1:q]
#pol = Meta.parse(pol) |> eval

"(5.00000000000000e0 .. 5.00000000000000e0) + (2.00000000000000e-1 .. 2.00000000000001e-1) * x + (-2.50000000000000e0 .. -2.50000000000000e0) * t + (-1.00000000000001e0 .. -1.00000000000000e0) * t * y + (-1.00000000000001e-1 .. -1.00000000000000e-1) * t * x + (-9.3750000" ⋯ 1408 bytes ⋯ "999999e0 .. 1.28500000000001e0) * t^4 * x * y + (2.30833333333333e-1 .. 2.30833333333335e-1) * t^4 * x^2 + (-8.41718750000027e0 .. -8.41718749999971e0) * t^5 * y + (5.42578124999991e0 .. 5.42578125000012e0) * t^5 * x + (-3.93610026041663e1 .. -3.93610026041660e1) * t^6 "

In [108]:
data_x

"(5.00000000000000e0 .. 5.00000000000000e0) + (2.00000000000000e-1 .. 2.00000000000001e-1) * x + (-2.50000000000000e0 .. -2.50000000000000e0) * t + (-1.00000000000001e0 .. -1.00000000000000e0) * t * y + (-1.00000000000001e-1 .. -1.00000000000000e-1) * t * x + (-9.3750000" ⋯ 1455 bytes ⋯ " (2.30833333333333e-1 .. 2.30833333333335e-1) * t^4 * x^2 + (-8.41718750000027e0 .. -8.41718749999971e0) * t^5 * y + (5.42578124999991e0 .. 5.42578125000012e0) * t^5 * x + (-3.93610026041663e1 .. -3.93610026041660e1) * t^6 + (-1.43728770330625e-8 .. 1.70313210117098e-8)"

In [32]:
data[3]

"local_t in [0 , 1.00000000000001e-2]"

In [38]:
dom_t = strip(split(data[3], "in")[2])
dom_t = replace(dom_t, "," => "..")
dom_t = replace(dom_t, "[" => "(")
dom_t = replace(dom_t, "]" => ")")
dom_t = Meta.parse(dom_t) |> eval

dom_x = strip(split(data[4], "in")[2])
dom_x = replace(dom_x, "," => "..")
dom_x = replace(dom_x, "[" => "(")
dom_x = replace(dom_x, "]" => ")")
dom_x = Meta.parse(dom_x) |> eval

dom_y = strip(split(data[4], "in")[2])
dom_y = replace(dom_y, "," => "..")
dom_y = replace(dom_y, "[" => "(")
dom_y = replace(dom_y, "]" => ")")
dom_y = Meta.parse(dom_y) |> eval

dom = IntervalBox(dom_t, dom_x, dom_y)

[0, 0.0100001] × [-1, 1] × [-1, 1]

In [39]:
typeof(dom)

IntervalBox{3, Float64}

In [41]:
x0 = IntervalBox(zeros(3)) # n + 1

[0, 0]³

In [42]:
# quedarme con el right-hand side
pol = split(data_x, "=")[2] |> strip

# renombrar variables
data_x = replace(data_x, "local_var_1" => "x")
data_x = replace(data_x, "local_var_2" => "y")
data_x = replace(data_x, "local_t" => "t")

# renombrar intervalos
data_x = replace(data_x, "," => "..")
data_x = replace(data_x, "[" => "(")
data_x = replace(data_x, "]" => ")")

data_x

TaylorModelN(pol, rem, x0, dom)

LoadError: MethodError: no method matching TaylorModelN(::Vector{SubString{String}}, ::Interval{Float64}, ::IntervalBox{3, Float64}, ::IntervalBox{3, Float64})
[0mClosest candidates are:
[0m  TaylorModelN([91m::TaylorN{T}[39m, ::Interval{S}, ::IntervalBox{N, S}, ::IntervalBox{N, S}) where {N, T, S} at /home/mforets/.julia/packages/TaylorModels/HzpNT/src/constructors.jl:127
[0m  TaylorModelN([91m::Interval{T}[39m, [91m::Integer[39m, ::IntervalBox{N, T}, ::IntervalBox{N, T}) where {N, T} at /home/mforets/.julia/packages/TaylorModels/HzpNT/src/constructors.jl:138
[0m  TaylorModelN([91m::Integer[39m, [91m::Integer[39m, ::IntervalBox{N, T}, ::IntervalBox{N, T}) where {N, T} at /home/mforets/.julia/packages/TaylorModels/HzpNT/src/constructors.jl:134
[0m  ...

### cosas viejas

In [ ]:
using Symbolics, IntervalArithmetic

In [ ]:
@variables local_t local_var_1 local_var_2

In [ ]:
f = eval(Meta.parse(data_x));

In [ ]:
expr = :([5.00000000000000e0 , 5.00000000000000e0] + [2.00000000000000e-1 , 2.00000000000001e-1] * local_var_1 + [-2.50000000000000e0 , -2.50000000000000e0] * local_t + [-1.00000000000001e0 , -1.00000000000000e0] * local_t * local_var_2 + [-1.00000000000001e-1 , -1.00000000000000e-1] * local_t * local_var_1 + [-9.37500000000000e0 , -9.37500000000000e0] * local_t^2 + [-4.00000000000002e-2 , -4.00000000000000e-2] * local_t * local_var_1 * local_var_2 + [-5.00000000000001e-1 , -5.00000000000000e-1] * local_t^2 * local_var_2 + [-1.37500000000001e0 , -1.37500000000000e0] * local_t^2 * local_var_1 + [2.39583333333333e0 , 2.39583333333334e0] * local_t^3 + [1.00000000000000e-1 , 1.00000000000001e-1] * local_t^2 * local_var_2^2 + [-1.20000000000001e-1 , -1.20000000000000e-1] * local_t^2 * local_var_1 * local_var_2 + [-4.00000000000002e-2 , -4.00000000000000e-2] * local_t^2 * local_var_1^2 + [3.79166666666666e0 , 3.79166666666668e0] * local_t^3 * local_var_2 + [-5.70833333333335e-1 , -5.70833333333333e-1] * local_t^3 * local_var_1 + [2.07421874999999e1 , 2.07421875000001e1] * local_t^4 + [4.00000000000000e-3 , 4.00000000000002e-3] * local_t^2 * local_var_1 * local_var_2^2 + [-4.00000000000002e-3 , -4.00000000000000e-3] * local_t^2 * local_var_1^2 * local_var_2 + [3.16666666666666e-1 , 3.16666666666668e-1] * local_t^3 * local_var_2^2 + [3.51666666666667e-1 , 3.51666666666668e-1] * local_t^3 * local_var_1 * local_var_2 + [-9.33333333333337e-2 , -9.33333333333334e-2] * local_t^3 * local_var_1^2 + [4.67708333333331e0 , 4.67708333333336e0] * local_t^4 * local_var_2 + [4.30885416666666e0 , 4.30885416666669e0] * local_t^4 * local_var_1 + [7.93619791666664e0 , 7.93619791666665e0] * local_t^5 + [-3.71654142564716e-6 , 3.61155996611113e-6])

In [ ]:
flow[4]

In [ ]:
flow[5]

In [ ]:
using TaylorModels

In Flow*, the reach-sets are represented using Taylor models in N+1 variables, where the last variable represents "time".

The text file `Lotka_Volterra.flow` contains the output generated by the algorithm.

Let's analyze in more detail the contents of this file.

The first part defines the symbols used for the state variables, in this case `x` and `y`. Also defined is the order of the Taylor models (in this case, `6`), the cutoff threshold (in this case, `1e-20`), and finally the models' name (in this case, `Lotka_Volterra`).

Header of `Lotka_Volterra.flow`:

```
state var x,y

gnuplot octagon x , y

order 6

cutoff 1.000000e-20

output Lotka_Volterra
```